# Extract Detected Changes in Sentinel-1 Imagery

Inspired by the following Google Earth Engine tutorials: 
- [Part 2 - Hypothesis testing](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-2); and
- [Part 3 - Multitemporal change detection](https://developers.google.com/earth-engine/tutorials/community/detecting-changes-in-sentinel-1-imagery-pt-3)

## Part 2 - Detecting Changes in Sentinel-1 Imagery

In [4]:
# Run the following cell to initialize the API. The output will contain instructions on how to grant this notebook access to Earth Engine using your account.
import ee

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()


Successfully saved authorization token.


In [51]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
import json
%matplotlib inline

In [52]:
#create function to download the image or imagecollection as you desire
def downloader(ee_object,region): 
    try:
        #download image
        if isinstance(ee_object, ee.image.Image):
            print('Its Image')
            url = ee_object.getDownloadUrl({
                    'scale': 10,
                    'crs': 'EPSG:4326',
                    'region': region
                })
            return url
        
        #download imagecollection
        elif isinstance(ee_object, ee.imagecollection.ImageCollection):
            print('Its ImageCollection')
            ee_object_new = ee_object.mosaic()
            url = ee_object_new.getDownloadUrl({
                    'scale': 10,
                    'crs': 'EPSG:4326',
                    'region': region
                })
            return url
    except:
        print("Could not download")

In [ ]:
with open('C:\\Desktop\\AI4ER\\03 - MRes\\Easter 2023\\MRes Project\\ArcticCCAM\\data\\satellite_imagery\\S1_links.csv', 'w', encoding='UTF8') as d:
writer.writerow(['filename', 'link'])
    for x in 

        #data
        with open('C:\\Desktop\\AI4ER\\03 - MRes\\Easter 2023\\MRes Project\\OneDrive - University of Cambridge\\geoJSON\\sample.json') as f:
            geoJSON = json.load(f)
            
        coords = geoJSON['features'][0]['geometry']['coordinates']
        aoi = ee.Geometry.Polygon(coords)
        
        im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                    .filterBounds(aoi)
                    .filterDate(ee.Date('2020-08-01'),ee.Date('2020-08-31'))
                    .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
                    .filter(ee.Filter.eq('relativeOrbitNumber_start', 15))
                    .sort('system:time_start'))
        
        import time
        acq_times = im_coll.aggregate_array('system:time_start').getInfo()
        [time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]
        
        im_list = im_coll.toList(im_coll.size())
        im1 = ee.Image(im_list.get(0)).select('VV').clip(aoi)
        im2 = ee.Image(im_list.get(1)).select('VV').clip(aoi)
        ratio = im1.divide(im2)
        m = 5
        
        # Decision threshold alpha/2:
        dt = f.ppf(0.0005, 2*m, 2*m)

        # LRT statistics.
        q1 = im1.divide(im2)
        q2 = im2.divide(im1)

        # Change map with 0 = no change, 1 = decrease, 2 = increase in intensity.
        c_map = im1.multiply(0).where(q2.lt(dt), 1)
        c_map = c_map.where(q1.lt(dt), 2)

        # Mask no-change pixels.
        c_map = c_map.updateMask(c_map.gt(0))

        region = aoi.toGeoJSONString()#region must in JSON format
        path = downloader(c_map,region)#call function
        print(path)#print the download URL
        
        writer.writerow([filename,path])